# Typical 1st & 2nd Order IIR-Filters in Audio Signal Processing

*This jupyter notebook is part of a [collection of notebooks](../index.ipynb) on various topics of Digital Signal Processing. Please direct questions and suggestions to [Sascha.Spors@uni-rostock.de](mailto:Sascha.Spors@uni-rostock.de).*

## Preliminaries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
np.set_printoptions(precision=16)  # print coeff in very high precision
fs = 48000  # sampling frequency in Hz

We use standard LaTex-style citing within the text. At the moment there is no commonly accepted long-term solution for rendering a reference list in a Jupyter notebook automatically, so we leave the bibtex information here as plain text as well. However, it's working with enabled `LaTeX environments for Jupyter` of the `Nbextensions` using the external `biblio.bib` file. Otherwise, digital sources appear linked with their URLs.

## Introduction

We recollect the most common analog transfer functions (Laplace domain) of 1st and 2nd order IIR audio filters and derive bilinear transforms (transfer functions in z-domain), applying prewarping for cut/mid frequency and for quality/bandwidth.

If you are interested in a Matlab based implementation of these filters, please have a look at the TU Berlin project [AKTools](https://www.ak.tu-berlin.de/menue/publications/open_research_tools/aktools/), cf. Fabian Brinkmann, Stefan Weinzierl: "AKtools -- An Open Software Toolbox for Signal Acquisition, Processing, and Inspection in Acoustics." In: *142nd AES Convention, Berlin, Germany, 2017*, e-Brief 309.

Although more sophisticated filter designs exist, the discussed filter characteristics solve many IIR based audio filtering tasks, such as in a typical digital audio workstation (DAW) and available [plugins](https://plugins.iem.at/), in digital mixing consoles, loudspeaker management systems, power amplifiers with DSP capabilities and audio player software.

Higher order IIR filters can be built from second order filters. A series connection of the discussed filter types achieves more complex transfer functions often being used in the mentioned applications. A so called graphical equalizer is a prominent example for that, which is however not restricted to build from series connection.

### Standardized vs. Non-Standardized Filter Characteristics
For the **standardized** lowpass, highpass, bandpass, bandstop and allpass filters, the Laplace transfer functions and their coefficients are well known from literature e.g. \cite{Moschytz1981}, [\cite{TietzeSchenk2002}](http://www.tietze-schenk.de/).

For shelving and parametric filters **no standardization w.r.t. the pole/zero qualities and cut-frequencies and thus positions of poles and zeros** exists. Based on \cite{Zoelzer2002,Zoelzer2005},[\cite{Zoelzer2008}](https://link.springer.com/chapter/10.1007/978-3-540-34301-1_15),[\cite{Zoelzer2011}](https://doi.org/10.1002/9781119991298) (in which these filters are derived with the help of allpass filters) and some other articles \cite{Kimball1938}, \cite{Bristow-Johnson}, [\cite{Bristow-Johnson1994}](http://www.aes.org/e-lib/browse.cfm?elib=6326), [\cite{Orfanidis1997}](http://www.aes.org/e-lib/browse.cfm?elib=7854), [\cite{Christensen2003}](http://www.aes.org/e-lib/browse.cfm?elib=12429) widely used approaches can be found.

\cite{Christensen2003} discusses the generalized form of the biquad transfer function for progammable filters.

Three characteristics (labeled as types **I, II and III**) are most prominent, leading to different transfer functions for the same parameter set. This regularly leads to misconceptions.

### Overview of Filter Transfer Functions

The follwing filter characteristics are discussed in this notebook:

[1st order lowpass](#lowpass1st)

[2nd order lowpass](#lowpass2nd)

[1st order highpass](#highpass1st) 

[2nd order highpass](#highpass2nd)

[2nd order bandpass](#bandpass2nd) 

[2nd order bandstop](#bandstop2nd)

[1st order allpass](#allpass1st) 

[2nd order allpass](#allpass2nd)

[2nd order PEQ](#peq2nd) type **I,I,III**

[1st order low-shelving](#lowshv1st) type **I,I,III**

[2nd order low-shelving](#lowshv2nd) type **I,I,III**

[1st order high-shelving](#highshv1st) type **I,I,III**

[2nd order low-shelving](#highshv2nd) type **I,I,III**

## Bilinear Transform of an IIR Biquad

The transformation of the analog filter prototypes (given as Laplace transfer function) towards the digital prototype (given as z-domain transfer function) is performed with the [bilinear transform](bilinear_transform.ipynb) \cite[Ch. 7.1.2]{OppenheimSchaferBuck2004}) applying frequency and bandwidth prewarping techniques.

There are other approaches such as the impulse invariant method, the matched z-transform method and higher order Taylor series truncations \cite{Clark2000}, \cite{Gunness2007}, \cite{Alaoui2007}, \cite{Schmidt2010}, \cite{Alaoui2010} that yield higher precision matching of the analog and digital transform. For didactical purpose we restrict ourselves to the bilinear transform of IIR biquads.

Let us define the angular frequency $\omega$ in rad/s, $\mathrm{i}^2=-1$ and the sampling frequency $f_s$ in Hz. The connection between the Laplace domain and the z-domain is given by

\begin{equation}
s = \mathrm{i}\,\omega \qquad z = \mathrm{e}^{\frac{s}{f_s}} \qquad \stackrel{\text{linearization from Taylor series}}{\rightarrow} \qquad s = 2\,f_s\,\frac{z-1}{z+1}.
\end{equation}

The continuous and discrete time transfer functions $H(s)$ and $H(z)$, respectively for the 2nd order filter (biquad) are given as 

\begin{equation}
H(s) = \frac{B_0\,s^2+B_1\,s+B_2}{A_0\,s^2+A_1\,s+A_2}
\qquad
H(z) = \frac{b_0+b_1\,z^{-1}+b_2\,z^{-2}}{1+a_1\,z^{-1}+a_2\,z^{-2}}
\end{equation}

following typical conventions for the denotation of the coefficients (Python and Matlab compatible).

The bilinear transform rule $s = 2\,f_s\,\frac{z-1}{z+1}$ can be inserted to $H(s)$ and the result can be brought to form of biquad given as $H(z)$ above. Then the coefficients for the biquad are generally given by, cf. [bilinear transform](bilinear_transform.ipynb)

\begin{align}
b_0 &= \frac{B_2+2\,B_1\,f_s+4\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
b_1 &= \frac{2\,B_2-8\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
b_2 &= \frac{B_2-2\,B_1\,f_s+4\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
a_1 &= \frac{2\,A_2-8\,A_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
a_2 &= \frac{A_2-2\,A_1\,f_s+4\,A_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}.
\end{align}

The function `bilinear_biquad()` below realizes the bilinear transformation based on the above given equations.

In [ ]:
def bilinear_biquad(B, A, fs = 48000):
    """get the bilinear transform of a 2nd-order Laplace transform

    bilinear transform H(s)->H(z) with s=2*fs*(z-1)/(z+1)
    
    input:    
    B[0]=B0   B[1]=B1   B[2]=B2 
    A[0]=A0   A[1]=A1   A[2]=A2
    fs...sampling frequency in Hz (default 48000)

           Y(s)   B0*s^2+B1*s+B2   B[0]*s^2+B[1]*s+B[2]
    H(s) = ---- = -------------- = --------------------
           X(s)   A0*s^2+A1*s+A2   A[0]*s^2+A[1]*s+A[2]

    output:
    b[0]=b0   b[1]=b1   b[2]=b2 
    a[0]=1    a[1]=a1   a[2]=a2

           Y(z)   b2*z^-2+b1*z^-1+b0   b[2]*z^-2+b[1]*z^-1+b[0]
    H(z) = ---- = ------------------ = ------------------------
           X(z)   a2*z^-2+a1*z^-1+ 1   a[2]*z^-2+a[1]*z^-1+a[0]  
    """
    a = np.array([0., 0., 0.])
    b = np.array([0., 0., 0.])
    fs2 = fs**2.
    
    tmp  = A[2] + 2*A[1]*fs + 4*A[0]*fs2
    b[0] = B[2] + 2*B[1]*fs + 4*B[0]*fs2

    b[1] = 2*B[2] - 8*B[0]*fs2
    a[1] = 2*A[2] - 8*A[0]*fs2

    b[2] = B[2] - 2*B[1]*fs + 4*B[0]*fs2
    a[2] = A[2] - 2*A[1]*fs + 4*A[0]*fs2

    a = a / tmp  # normalize such that a[0]=1
    b = b / tmp
    a[0] = 1.  # set as double
    
    return b, a

## Frequency and Bandwidth Prewarping

A so called prewarping \cite[eq. (7.28)]{OppenheimSchaferBuck2004} of the cut/mid frequency $f_c$/$f_m$ in Hz

\begin{equation}
\omega_{c} \Leftarrow 2\,f_s\,\tan{\left(\pi\frac{f_{c}}{f_s}\right)},
\end{equation}

\begin{equation}
\omega_{m} \Leftarrow 2\,f_s\,\tan{\left(\pi\frac{f_{m}}{f_s}\right)}
\end{equation}

is realized in the function `prewarping_f()`.

In [ ]:
def prewarping_f(f, fs = 48000):
    """do the frequency prewarping
    
    input:
    f...analog frequency in Hz to be prewarped
    fs...sampling frequency in Hz
    
    output:
    prewarped digital angular frequency in rad/s
    """
    return 2*fs*np.tan(np.pi*f/fs)

The connection between the bandwidth $BW_\text{oct}$ in octaves and the corresponding bandpass-related quality $Q_\text{BP}$ is given as

\begin{equation}
BW_\text{oct} = \frac{2}{\log_\mathrm{e}(2)}\,\sinh^{-1}(\frac{1}{2\,Q_\text{BP}})
\qquad
Q_\text{BP} = \frac{1}{2\,\sinh(\frac{\log_\mathrm{e}(2)}{2}\,BW_\text{oct})}
\end{equation}

with the natural logarithm $\log_\mathrm{e}(\cdot)$ and the hyperbolic sine $\sinh(\cdot)$.

In [ ]:
def bw_from_q(QBP):
    """convert bandpass quality to bandwidth in octaves"""
    return 2/np.log(2) * np.arcsinh(1/(2*QBP))
    
def q_from_bw(BWoct):
    """convert bandwidth in ocatves to bandpass quality"""
    return 1 / (2*np.sinh(np.log(2)/2*BWoct))

The prewarping of the bandpass quality for the parametric, bandpass and bandstop filters can be done with

* **tan**-prewarping, cf. [\cite{Clark2000}](http://www.aes.org/e-lib/browse.cfm?elib=12069)

\begin{equation}
Q_\text{BP} \Leftarrow \frac{\frac{\pi\,f_m}{f_s}}{\tan\left(\frac{\pi\,f_m}{f_s}\right)} Q_\text{BP}
\end{equation}

* **cos**-prewarping, cf. \cite{Thaden1997}

\begin{equation}
Q_\text{BP} \Leftarrow \cos(\pi\,\frac{f_m}{f_s}) Q_\text{BP}
\end{equation}

* **sin**-prewarping of the bandwidth in octaves, cf. \cite{Bristow-Johnson1994}

\begin{equation}
BW_\text{oct} \Leftarrow \frac{2\,\pi\,\frac{f_m}{f_s}}{\sin(2\,\pi\,\frac{f_m}{f_s})} BW_\text{oct}
\end{equation}

exhibiting slightly different characteristics.


In [ ]:
def prewarping_q(Q, fm, fs = 48000, WarpType = "cos"):
    """do the quality prewarping
    
    input:
    Q...bandpass quality to be prewarped
    fm...analog mid-frequency in Hz
    fs...sampling frequency in Hz
    WarpType:
    "sin"...Robert Bristow-Johnson (1994): "The equivalence of various methods
    of computing biquad coefficients for audio parametric equalizers."In:
    Proc. of 97th AES Convention, San Fransisco. eq. (14)
    "cos"...Rainer Thaden (1997): "Entwicklung und Erprobung einer digitalen 
    parametrischen Filterbank." Diplomarbeit, RWTH Aachen
    "tan"...Clark, R.J.; Ifeachor, E.C.; Rogers, G.M.; et al. (2000):
    "Techniques for Generating Digital Equalizer Coefficients".
    In: J. Aud. Eng. Soc. 48(4):281-298.
    
    output:
    prewarped quality
    """    
    if WarpType == "sin":
        BW = bw_from_q(Q)
        w0 = 2*np.pi*fm / fs
        BW = BW*w0 / np.sin(w0)
        Qp = q_from_bw(BW)        
    elif WarpType == "cos":
        Qp = Q * np.cos(np.pi*fm / fs)
    elif WarpType == "tan":
        Qp = Q * (np.pi*fm / fs) / np.tan(np.pi*fm / fs)
    else:
        Qp = Q
    return Qp

## Plotting Routine
We establish a plotting routine for the `bode_plot()`, i.e. the magnitude and phase spectrum of the filter's transfer function. For that we use the `freqs()` and `freqz()` functions implemented in the `signal` package. We also use the self-implemented `zplane()` function for plotting zeros and poles in the z-plane.

In [ ]:
from matplotlib.markers import MarkerStyle
from matplotlib.patches import Circle 

def zplane_plot(ax, z, p):
    """realize a zplane plot
    
    input:
    ax...axes handle
    z...zeros
    p...poles
    output:
    zplane plot into ax
    """
    if ax is None:
       ax = plt.gca()    
    
    ax.plot(np.real(z), np.imag(z), 
            'o', label = "zeros", 
            color = 'C2', fillstyle = 'none', 
            markersize = 15, markeredgewidth = 3)
    ax.plot(np.real(p), np.imag(p),
            'x', label = "poles",
            color = 'C3', fillstyle = 'none',
            markersize = 15, markeredgewidth = 3)
    ax.axvline(0, color = '0.7')
    ax.axhline(0, color = '0.7')
    unit_circle = Circle((0,0), radius = 1, fill = False,
                         color = 'black', linestyle = '-', alpha = 0.9)
    ax.add_patch(unit_circle)
    ax.set_xscale("linear")
    ax.set_yscale("linear")
    ax.set_xlabel(r'Real{$z$}', color = 'xkcd:navy blue')
    ax.set_ylabel(r'Imag{$z$}', color = 'xkcd:navy blue')
    ax.set_title("Poles x and zeros o of discrete-time domain filter",
                 color = 'xkcd:navy blue')
    ax.axis("equal")
    ax.set_xlim(-1.25, +1.25)
    ax.set_xticks(np.arange(-1.25, 1.25+0.25, 0.25))
    ax.set_xticklabels(["-1.25", "-1", "-0.75", "-0.5", "-0.25", "0",
                        "0.25", "0.5", "0.75", "1", "1.25"], 
                       color = 'xkcd:navy blue')
    ax.set_ylim(-1.25, +1.25)
    ax.set_yticks(np.arange(-1.25, 1.25+0.25, 0.25))
    ax.set_yticklabels(["-1.25", "-1", "-0.75", "-0.5", "-0.25", "0",
                        "0.25", "0.5", "0.75", "1", "1.25"], 
                       color = 'xkcd:navy blue')
    ax.legend(loc = "best")
    ax.grid(True, which = "both", axis = "both",
             linestyle = "-", linewidth = 0.5,
             color = (0.8, 0.8, 0.8))    

In [ ]:
def bode_plot(fig, B, A, b, a, fs, N = 2**12):
    """realize a bode plot containing magnitude, phase and zplane
    
    input:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    fs...sampling frequency in Hz
    output:
    bode plot as new figure
    """
    if fig is None:
        fig = plt.figure(figsize = (16,9))
        
    p = np.roots(a)
    z = np.roots(b)
    W, Hd = sig.freqz(b, a, N)
    s, Ha = sig.freqs(B, A, fs*W)
    if Hd[0] == 0:
        Hd[0] = 1e-15  # avoid zero at DC for plotting dB
    if Ha[0] == 0:
        Ha[0] = 1e-15
    f = fs*W / (2*np.pi)
    
    gs = fig.add_gridspec(2, 2)
    # magnitude
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(f, 20*np.log10(np.abs(Ha)), 'C0', 
             label = r'$|H(\omega)|$ continuous-time',
             linewidth = 3)    
    ax1.plot(f, 
             20*np.log10(np.abs(Hd)), 
             'C1',
             label = (r'$|H(\Omega)|$ discrete-time, fs=%5.f Hz' %fs),
             linewidth = 2)
    ax1.set_xscale("log")
    ax1.set_yscale("linear")
    ax1.set_xlabel(r'$f$ / Hz', color = 'xkcd:navy blue')
    ax1.set_ylabel(r'$A$ / dB', color = 'xkcd:navy blue')
    ax1.set_title("Bode plot: magnitude", color = 'xkcd:navy blue')
    ax1.set_xlim(20, 20000)
    ax1.set_xticks((20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000))
    ax1.set_xticklabels(["20", "50",
                         "100", "200", "500",
                         "1k", "2k", "5k",
                         "10k", "20k"], color = 'xkcd:navy blue')
    ax1.set_ylim(-15, 15)
    ax1.set_yticks(np.arange(-15, 15+3, 3))
    ax1.set_yticklabels(["-15", "-12", "-9", "-6", "-3", "0",
                         "3", "6", "9", "12", "15"],
                        color = 'xkcd:navy blue')
    ax1.legend(loc = "lower left")
    ax1.grid(True, which = "both", axis = "both",
             linestyle = "-", linewidth = 0.5,
             color = (0.8, 0.8, 0.8))    

    # phase
    ax2 = fig.add_subplot(gs[1, 0])  
    ax2.plot(f, (np.angle(Ha)*180/np.pi), 'C0',
             label = r'$\mathrm{angle}(H(\omega))$ continuous-time',
             linewidth = 3)
    ax2.plot(f, 
             (np.angle(Hd)*180/np.pi), 
             'C1',
             label = (r'$\mathrm{angle}(H(\Omega))$ discrete-time, fs=%5.f Hz' %fs),
             linewidth = 2) 
    ax2.set_xscale("log")
    ax2.set_yscale("linear")
    ax2.set_xlabel(r'$f$ / Hz', color = 'xkcd:navy blue')
    ax2.set_ylabel(r'$\phi$ / deg', color = 'xkcd:navy blue')
    ax2.set_title("Bode plot: phase", color = 'xkcd:navy blue')
    ax2.set_xlim(20, 20000)
    ax2.set_xticks((20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000))
    ax2.set_xticklabels(["20", "50",
                         "100", "200", "500",
                         "1k", "2k", "5k",
                         "10k", "20k"], color = 'xkcd:navy blue')
    ax2.set_ylim(-180, +180)
    ax2.set_yticks(np.arange(-180, 180+45, 45))
    ax2.set_yticklabels(["-180", "-135", "-90", "-45", "0",
                         "45", "90", "135", "180"],
                        color = 'xkcd:navy blue')
    ax2.legend(loc = "lower left")
    ax2.grid(True, which = "both", axis = "both",
             linestyle = "-", linewidth = 0.5,
             color = (0.8, 0.8, 0.8))
    
    # zplane
    ax3 = fig.add_subplot(gs[:, 1])  
    zplane_plot(ax3, z, p)   

## Example

### Without Frequency Prewarping
A first example demonstrates the application of the bilinear transform and plotting routine. We use a lowpass filter (simplest RC circuit) with the cut frequency $f_c$ in Hz, which is linked to $\omega_c=2\,\pi\,f_c=\frac{1}{R\,C}$. For the discrete time filter we need to specify the sampling frequency $f_s$ in Hz.

In [ ]:
# example lowpass 1st order without frequency pre-warping
fc = 10000  # -3dB cut frequency in Hz
wg = 2*np.pi*fc  # rad/s

B = np.array([0., 0., 1.])  # Laplace transfer function
A = np.array([0., 1./wg, 1.])

b, a = bilinear_biquad(B, A, fs)  # z-transfer function
bsig, asig = sig.bilinear(B, A, fs)

# check if equivalent
np.testing.assert_allclose(b, bsig)
np.testing.assert_allclose(a, asig)

bode_plot(None, B, A, b, a, fs)

### With Frequency Prewarping

Now, do the same with frequency prewarping, in order that the intended 'analog' $f_c$ matches the 'digital' $f_c$ in discrete time domain.

In [ ]:
# example lowpass 1st order with frequency pre-warping
fc = 10000
wc = 2*np.pi*fc
B = np.array([0., 0., 1.])
A = np.array([0., 1./wc, 1.])

wp = prewarping_f(fc, fs)

Bp = np.array([0., 0., 1.])
Ap = np.array([0., 1./wp, 1.])
b, a = bilinear_biquad(Bp, Ap, fs) 
bode_plot(None, B, A, b, a, fs)

## Definition of 1st / 2nd Order Filter Transfer Functions

Below we implement functions for lowpass, highpass, allpass, bandpass, bandstop, shelving and parametric filters. These characteristics are very often used in audio applications, cf. e.g. [\cite{TietzeSchenk2002}](http://www.tietze-schenk.de/), \cite{Moschytz1981}, \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}, \cite{Bristow-Johnson1994}

In [ ]:
def biquad_lp1st(fc, fs):  
    """calc coeff for lowpass 1st order
    
    input:
    fc...cut frequency in Hz
    fs...sampling frequency in Hz
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)
    B = np.array([0., 
                  0., 
                  1.])
    A = np.array([0., 
                  1. / wc,
                  1.])
    Bp = np.array([0., 
                   0., 
                   1.])
    Ap = np.array([0., 
                   1. / wp,
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_lp2nd(fc, fs, bi, ai):
    """calc coeff for lowpass 2nd order
    
    input:
    fc...cut frequency in Hz
    fs...sampling frequency in Hz
    bi, ai...filter characteristics coefficients, e.g.
    bi = 0.6180, ai = 1.3617 for Bessel
    bi = 1, ai = 1.4142 for Butterworth    
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """    
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)
    B = np.array([0., 
                  0., 
                  1.])
    A = np.array([bi / (wc**2), 
                  ai / wc, 
                  1.])
    Bp = np.array([0., 
                   0., 
                   1.])
    Ap = np.array([bi / (wp**2), 
                   ai / wp, 
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_hp1st(fc, fs):
    """calc coeff for highpass 1st order
    
    input:
    fc...cut frequency in Hz
    fs...sampling frequency in Hz
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """     
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)
    B = np.array([0., 
                  1. / wc, 
                  0.])
    A = np.array([0.,
                  1. / wc,
                  1.])
    Bp = np.array([0., 
                   1. / wp,
                   0.])
    Ap = np.array([0.,
                   1. / wp,
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_hp2nd(fc, fs, bi, ai):  
    """calc coeff for highpass 2nd order
    
    input:
    fc...cut frequency in Hz
    fs...sampling frequency in Hz
    bi, ai...filter characteristics coefficients, e.g.
    bi = 0.6180, ai = 1.3617 for Bessel
    bi = 1, ai = 1.4142 for Butterworth       
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """  
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)
    B = np.array([1. / (wc**2),
                  0.,
                  0.])
    A = np.array([1. / (wc**2),
                  ai / wc,
                  bi])
    Bp = np.array([1. / (wp**2),
                   0.,
                   0.])
    Ap = np.array([1. / (wp**2),
                   ai / wp,
                   bi])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_bp2nd(fm, Q, fs, QWarpType):
    """calc coeff for bandpass 2nd order
    
    input:
    fm...mid frequency in Hz
    Q...bandpass quality
    fs...sampling frequency in Hz
    QWarpType..."sin", "cos", "tan"
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """      
    wm = 2*np.pi*fm
    wp = prewarping_f(fm, fs)
    Qp = prewarping_q(Q, fm, fs, QWarpType)
    B = np.array([0., 
                  1. / (Q*wm),
                  0.])
    A = np.array([1. / (wm**2),
                  1. / (Q*wm),
                  1.])
    Bp = np.array([0.,
                   1. / (Qp*wp),
                   0.])
    Ap = np.array([1. / (wp**2),
                   1. / (Qp*wp),
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_bs2nd(fm, Q, fs, QWarpType):  
    """calc coeff for bandstop 2nd order
    
    input:
    fm...mid frequency in Hz
    Q...bandpass quality
    fs...sampling frequency in Hz
    QWarpType..."sin", "cos", "tan"
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """     
    wm = 2*np.pi*fm
    wp = prewarping_f(fm, fs)
    Qp = prewarping_q(Q, fm, fs, QWarpType)
    B = np.array([1. / wm**2,
                  0.,
                  1.])
    A = np.array([1. / wm**2,
                  1. / (Q*wm),
                  1.])
    Bp = np.array([1. / wp**2,
                   0.,
                   1.])
    Ap = np.array([1. / wp**2,
                   1. / (Qp*wp),
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a   



def biquad_ap1st(fc, ai, fs):
    """calc coeff for allpass 1st order
    
    input:
    fc...cut frequency in Hz
    ai...filter characteristics coefficients, e.g. ai = 1
    fs...sampling frequency in Hz
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """    
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)
    B = np.array([0.,
                  -ai / wc,
                  1.])
    A = np.array([0.,
                  +ai / wc,
                  1.])
    Bp = np.array([0.,
                   -ai / wp,
                   1.])
    Ap = np.array([0.,
                   +ai / wp,
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_ap2nd(fc, bi, ai, fs):
    """calc coeff for allpass 2nd order
    
    input:
    fc...cut frequency in Hz
    bi, ai...filter characteristics coefficients, e.g.
    bi = 1, ai = 1.4142 
    fs...sampling frequency in Hz
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """     
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)    
    B = np.array([bi / (wc**2),
                  -ai / wc,
                  1.])
    A = np.array([bi / (wc**2),
                  +ai / wc,
                  1.])
    Bp = np.array([bi/(wp**2.),
                   -ai/wp,
                   1.])
    Ap = np.array([bi/(wp**2.),
                   +ai/wp,
                   1.])
    b, a = bilinear_biquad(Bp, Ap, fs)
    return B, A, b, a



def biquad_peq2nd(fm, G, Q, fs, PEQType, QWarpType):
    """calc coeff for peak/dip equalizer (PEQ) 2nd order
    
    input:
    fm...mid frequency in Hz
    G...gain or attenuation in dB
    Q...quality
    fs...sampling frequency in Hz
    PEQType..."I", "II", "III"
    QWarpType..."sin", "cos", "tan"    
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """    
    wm = 2*np.pi*fm
    wp = prewarping_f(fm, fs)
    g = 10**(G/20)
    Qp = prewarping_q(Q, fm, fs, QWarpType)    
    if PEQType == "I":  # aka constant-Q PEQ
        gamma = g
        delta = g
    elif PEQType == "II":  # aka symmetrical PEQ
        gamma = 1.
        delta = g
    elif PEQType == 'III':  # aka one-half pad loss PEQ
        gamma = g**0.5
        delta = g**0.5
    else:
        gamma = unknown_PEQType
        delta = unknown_PEQType
    if G > 0.:
        B = np.array([1. / wm**2,
                      delta / (Q*wm),
                      1.])
        A = np.array([1. / wm**2,
                      (delta/g) / (Q*wm),
                      1.])
        Bp = np.array([1. / wp**2,
                       delta / (Qp*wp),
                       1.])
        Ap = np.array([1. / wp**2,
                       (delta/g) / (Qp*wp),
                       1.])          
    else:
        B = np.array([1. / wm**2,
                      gamma / (Q*wm),
                      1.])
        A = np.array([1. / wm**2,
                      (gamma/g) / (Q*wm),
                      1.])
        Bp = np.array([1. / wp**2,
                       gamma / (Qp*wp),
                       1.])
        Ap = np.array([1. / wp**2,
                       (gamma/g) / (Qp*wp),
                       1.])    
    b, a = bilinear_biquad(Bp, Ap, fs)
    
    # if G==0 dB make filter flat    
    if np.isclose(G, 0., rtol = 1e-05, atol = 1e-08, equal_nan = False):
        B = np.array([0., 0., 1.]) 
        A = np.array([0., 0., 1.])         
        b = np.array([1., 0., 0.])
        a = np.array([1., 0., 0.])     
    
    return B, A, b, a       



def biquad_lshv1st(fc, G, fs, ShvType):
    """calc coeff for lowshelving 1st order
    
    input:
    fc...cut frequency in Hz
    G...gain or attenuation in dB
    fs...sampling frequency in Hz
    ShvType..."I", "II", "III"
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """        
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs) 
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25
    else:
        alpha = unknown_ShvType
    if G > 0.:
        B = np.array([0.,
                      1. / wc,
                      g * alpha**-2])
        A = np.array([0.,
                      1. / wc,
                      alpha**-2])
        Bp = np.array([0.,
                       1. / wp,
                       g * alpha**-2])
        Ap = np.array([0.,
                       1. / wp,
                       alpha**-2])         
    else:
        B = np.array([0.,
                      1. / wc,
                      alpha**2])
        A = np.array([0.,
                      1. / wc,
                      g**-1 * alpha**2]) 
        Bp = np.array([0.,
                       1. / wp,
                       alpha**2])
        Ap = np.array([0.,
                       1. / wp,
                       g**-1 * alpha**2])        
    b, a = bilinear_biquad(Bp, Ap, fs)
    
    # if G==0 dB make filter flat 
    if np.isclose(G, 0., rtol = 1e-05, atol = 1e-08, equal_nan = False):
        B = np.array([0., 0., 1.])
        A = np.array([0., 0., 1.])         
        b = np.array([1., 0., 0.])
        a = np.array([1., 0., 0.])     
    
    return B, A, b, a  



def biquad_lshv2nd(fc, G, Qz, Qp, fs, ShvType):
    """calc coeff for lowshelving 2nd order
    
    input:
    fc...cut frequency in Hz
    G...gain or attenuation in dB
    Qz...zero Quality, e.g. Qz = 1./np.sqrt(2.) for Butterworth quality
    Qp...pole quality, e.g. Qp = 1./np.sqrt(2.) for Butterworth quality    
    fs...sampling frequency in Hz
    ShvType..."I", "II", "III"
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """       
    g = 10**(G/20)
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)      
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25       
    else:
        alpha = unknown_ShvType
    if G > 0.:
        B = np.array([1. / wc**2, 
                      g**0.5 * alpha**-1 / (Qz*wc), 
                      g * alpha**-2])
        A = np.array([1. / wc**2, 
                      alpha**-1 / (Qp*wc), 
                      alpha**-2]) 
        Bp = np.array([1. / wp**2,
                       g**0.5 * alpha**-1 / (Qz*wp),
                       g * alpha**-2])
        Ap = np.array([1. / wp**2,
                       alpha**-1 / (Qp*wp),
                       alpha**-2])         
    else:
        B = np.array([1. / wc**2, 
                      alpha / (Qz*wc), 
                      alpha**2])
        A = np.array([1. / wc**2, 
                      g**-0.5 * alpha / (Qp*wc), 
                      g**-1 * alpha**2])
        Bp = np.array([1. / wp**2, 
                       alpha / (Qz*wp),
                       alpha**2])
        Ap = np.array([1. / wp**2,
                       g**-0.5 * alpha / (Qp*wp),
                       g**-1 * alpha**2])        
    b, a = bilinear_biquad(Bp, Ap, fs)
    
    # if G==0 dB make filter flat 
    if np.isclose(G, 0., rtol = 1e-05, atol = 1e-08, equal_nan = False):  
        B = np.array([0., 0., 1.])
        A = np.array([0., 0., 1.])         
        b = np.array([1., 0., 0.])
        a = np.array([1., 0., 0.])     
    
    return B, A, b, a  



def biquad_hshv1st(fc, G, fs, ShvType):
    """calc coeff for highshelving 1st order
    
    input:
    fc...cut frequency in Hz
    G...gain or attenuation in dB
    fs...sampling frequency in Hz
    ShvType..."I", "II", "III"
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """     
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)  
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25
    else:
        alpha = unknown_ShvType
    if G > 0.:
        B = np.array([0., 
                      g * alpha**-2 / wc,
                      1.])
        A = np.array([0., 
                      alpha**-2 / wc,
                      1.])
        Bp = np.array([0., 
                       g * alpha**-2 / wp,
                       1.])
        Ap = np.array([0., 
                       alpha**-2 / wp,
                       1.])         
    else:
        B = np.array([0., 
                      alpha**2 / wc,
                      1.])
        A = np.array([0., 
                      g**-1 * alpha**2 / wc,
                      1.])
        Bp = np.array([0., 
                       alpha**2 / wp,
                       1.])
        Ap = np.array([0.,
                       g**-1 * alpha**2 / wp,
                       1.])        
    b, a = bilinear_biquad(Bp, Ap, fs)

    # if G==0 dB make filter flat 
    if np.isclose(G, 0., rtol = 1e-05, atol = 1e-08, equal_nan = False):  
        B = np.array([0., 0., 1.])
        A = np.array([0., 0., 1.])         
        b = np.array([1., 0., 0.])
        a = np.array([1., 0., 0.])     
    
    return B, A, b, a  



def biquad_hshv2nd(fc, G, Qz, Qp, fs, ShvType):
    """calc coeff for highshelving 2nd order
    
    input:
    fc...cut frequency in Hz
    G...gain or attenuation in dB
    Qz...zero Quality, e.g. Qz = 1./np.sqrt(2.) for Butterworth quality
    Qp...pole quality, e.g. Qp = 1./np.sqrt(2.) for Butterworth quality    
    fs...sampling frequency in Hz
    ShvType..."I", "II", "III"
    output:
    B...numerator cofficients Laplace transfer function
    A...denominator cofficients Laplace transfer function
    b...numerator coefficients z-transfer function
    a...denominator cofficients z-transfer function
    """ 
    wc = 2*np.pi*fc
    wp = prewarping_f(fc, fs)      
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25       
    else:
        alpha = unknown_ShvType
    if G > 0.:
        B = np.array([g * alpha**-2 / wc**2,
                      g**0.5 * alpha**-1 / (Qz*wc),
                      1.])
        A = np.array([alpha**-2 / wc**2,
                      alpha**-1 / (Qp*wc),
                      1.])
        Bp = np.array([g * alpha**-2 / wp**2,
                       g**0.5 * alpha**-1 / (Qz*wp),
                       1.])
        Ap = np.array([alpha**-2 / wp**2,
                       alpha**-1 / (Qp*wp),
                       1.])          
    else:
        B = np.array([alpha**2 / wc**2,
                      alpha / (Qz*wc),
                      1.])
        A = np.array([g**-1 * alpha**2 / wc**2,
                      g**-0.5 * alpha / (Qp*wc),
                      1.])    
        Bp = np.array([alpha**2 / wp**2,
                       alpha / (Qz*wp),
                       1.])
        Ap = np.array([g**-1 * alpha**2 / wp**2,
                       g**-0.5 * alpha/(Qp*wp),
                       1.])     
    b, a = bilinear_biquad(Bp, Ap, fs)
    
    # if G==0 dB make filter flat 
    if np.isclose(G, 0., rtol = 1e-05, atol = 1e-08, equal_nan = False):
        B = np.array([0., 0., 1.])
        A = np.array([0., 0., 1.])         
        b = np.array([1., 0., 0.])
        a = np.array([1., 0., 0.])     
    
    return B, A, b, a               

<a id="lowpass1st"></a>
## 1st Order Lowpass

In [ ]:
fc = 1000  # cut frequency in Hz
B, A, b, a = biquad_lp1st(fc, fs)
bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="lowpass2nd"></a>
## 2nd Order Lowpass

In [ ]:
fc = 1000  # Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2)  # here for Butterworth
B, A, b, a = biquad_lp2nd(fc, fs, bi, ai)
bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highpass1st"></a>
## 1st Order Highpass

In [ ]:
fc = 1000  # Hz
B, A, b, a = biquad_hp1st(fc, fs)
bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highpass2nd"></a>
## 2nd Order Highpass

In [ ]:
fc = 1000  # Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2)  # here for Butterworth
B, A, b, a = biquad_hp2nd(fc, fs, bi, ai)
bode_plot(None, B, A, b, a, fs)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="bandpass2nd"></a>
## 2nd Order Bandpass

In [ ]:
BW = 2  # bandwidth in octaves
fm = 1000  # mid frequency in Hz

flow =  2**(-BW/2) * fm  # lower cut  (-3 dB) frequency in Hz 
fhigh = 2**(+BW/2) * fm  # higher cut (-3 dB) frequency in Hz 
QBP = q_from_bw(BW)
np.testing.assert_allclose(QBP, fm / (fhigh-flow))
QWarpType = "cos" # sin, cos, tan
B, A, b, a = biquad_bp2nd(fm, QBP, fs, QWarpType)
bode_plot(None, B, A, b, a, fs)

print("flow=", flow, "Hz")
print("fmid=", fm, "Hz")
print("fhigh=", fhigh, "Hz")
print("QBP=", QBP)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="bandstop2nd"></a>
## 2nd Order Bandstop

In [ ]:
fm = 1000  # Hz
QBP = 2/3
QWarpType = "cos"
B, A, b, a = biquad_bs2nd(fm, QBP, fs, QWarpType)
bode_plot(None, B, A, b, a, fs)

print("flow=", flow, "Hz")
print("fmid=", fm, "Hz")
print("fhigh=", fhigh, "Hz")
print("QBP=", QBP)
print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="allpass1st"></a>
## 1st Order Allpass

In [ ]:
fc = 1000  # Hz
ai = 1  # quality coeff
B, A, b, a = biquad_ap1st(fc, ai, fs)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="allpass2nd"></a>
## 2nd Order Allpass

In [ ]:
fc = 1000  #  Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2) # here for Butterworth like
B, A, b, a = biquad_ap2nd(fc, bi, ai, fs)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="peq2nd"></a>
## 2nd Order PEQ

In [ ]:
BW = 2  # bandwidth in octaves
fm = 1000  # Hz
G = 12  # gain in dB
Q = q_from_bw(BW)
PEQType = "III" # I,II,III
QWarpType = "cos"
B, A, b, a = biquad_peq2nd(fm, G, Q, fs, PEQType, QWarpType)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="lowshv1st"></a>
## 1st Order Low-Shelving

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
B, A, b, a = biquad_lshv1st(fc, G, fs, ShvType)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="lowshv2nd"></a>
## 2nd Order Low-Shelving

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
Qz = 1/np.sqrt(2)  # Butterworth quality
Qp = Qz
B, A, b, a = biquad_lshv2nd(fc, G, Qz, Qp, fs, ShvType)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highshv1st"></a>
## 1st Order High-Shelving

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
B, A, b, a = biquad_hshv1st(fc, G, fs, ShvType)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

<a id="highshv2nd"></a>
## 2nd Order High-Shelving

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
Qz = 1/np.sqrt(2)  # Butterworth quality
Qp = Qz
B, A, b, a = biquad_hshv2nd(fc, G, Qz, Qp, fs, ShvType)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

In [ ]:
fc = 1000  # Hz
G = 12  # dB
ShvType = "III"
Qz = np.sqrt(2)
Qp = Qz
B, A, b, a = biquad_hshv2nd(fc, G, Qz, Qp, fs, ShvType)
bode_plot(None, B, A, b, a, fs)

print("B=", B)
print("A=", A)
print("b=", b)
print("a=", a)

## Exercise

1.) Check the differences of the types I,II vs. III for the PEQ and shelving filters w.r.t. their cut frequencies.

Hints: 

* The **type III** filters as for PEQs and shelving filters consistently define the cut frequency as the point of half gain in dB.

* Can you define the half gain $G/2$ cut frequencies of type III for small gain/attenuation $|G| <3$ dB?

* Can you define the '3 dB cut frequencies' for small gain/attenuation $|G| <3$ dB for type I and II?

* Where are the '3dB cut frequencies' located for type I,II for large gain or large attenuation? 

* Check how the filters of your favorite audio software are implemented.

2.) Discuss the differences of the sin, cos, tan-style quality pre-warping. For very high frequencies and rather small bandwidths the differences become obvious. 

In [ ]:
# high shelve example
Qz = 1/np.sqrt(2)
Qp = Qz
fc = 1000  # Hz
G = 15  # dB

# fc at 3 dB
B, A, b, a = biquad_hshv2nd(fc, G, Qz, Qp, fs, "II")
bode_plot(None, B, A, b, a, fs)

# fc at 7.5 dB
B, A, b, a = biquad_hshv2nd(fc, G, Qz, Qp, fs, "III")
bode_plot(None, B, A, b, a, fs)

# fc at 12 dB (15 dB - 3 dB)
B, A, b, a = biquad_hshv2nd(fc, G, Qz, Qp, fs, "I")
bode_plot(None, B, A, b, a, fs)

In [ ]:
# PEQ example
BW = 2  # bandwidth in octaves
fm = 2000  # mid frequency in Hz; using fm=1000 and BW=2, then fc is at 1 kHz
G = 15  # dB
QBP = q_from_bw(BW)
QWarpType = "cos"

B, A, b, a = biquad_peq2nd(fm, G, QBP, fs, "II", QWarpType)
bode_plot(None, B, A, b, a, fs)

B, A, b, a = biquad_peq2nd(fm, G, QBP, fs, "III", QWarpType)
bode_plot(None, B, A, b, a, fs)

# note: this type is not symmetrical w.r.t. 
#the same dB amount of gain or attenuation
B, A, b, a = biquad_peq2nd(fm, G, QBP, fs, "I", QWarpType)
bode_plot(None, B, A, b, a, fs)

## Excursus: Higher Order Filters

In audio signal processing also IIR filters of higher order are applied. However, they will be usually split into second order sections for series connection. Note that the group delay of an IIR filter increases with higher filter order. 

A prominent example of using higher order IIR filters is found in loudspeaker design. For a multi-way loudspeaker each driver must only be driven within an appropriate intended frequency range. This is usually realized with a bandpass filter that is built from a higher order lowpass and a higher order highpass filter. Thus, each loudspeaker exhibits its own bandpass filter. The acoustic summation of all driver's generated sound pressure then yields the desired full audio bandwidth.

Consider, the following two-driver application, in which the same bandpass characteristic is used for the low and high frequency driver resulting up in a 4th order so called Linkwitz-Riley frequency crossover.

In [ ]:
N = 2**14  # frequency resolution

# two equal 2nd order Butterworth filters in series
# is known as a 4th order Linkwitz-Riley filter
bi1 = 1 
ai1 = np.sqrt(2)
bi2 = 1 
ai2 = np.sqrt(2)

# low frequency driver range
fLow_LP = 1000
fLow_HP = 20

# high frequency driver range
fHigh_LP = 20000
fHigh_HP = fLow_LP

# note that Laplace domain filters are actually used in the code below,
# since z-domain is always overwritten 
# get transfer functions of LOW frequency driver's bandpass
# highpass
B, A, b, a = biquad_hp2nd(fLow_HP, fs, bi1, ai1)
W, LowHP1 = sig.freqz(b, a, N)
s, LowHP1 = sig.freqs(B, A, fs*W)
B, A, b, a = biquad_hp2nd(fLow_HP, fs, bi2, ai2)
W, LowHP2 = sig.freqz(b, a, N)
s, LowHP2 = sig.freqs(B, A, fs*W)
# lowpass
B, A, b, a = biquad_lp2nd(fLow_LP, fs, bi1, ai1)
W, LowLP1 = sig.freqz(b, a, N)
s, LowLP1 = sig.freqs(B, A, fs*W)
B, A, b, a = biquad_lp2nd(fLow_LP, fs, bi2, ai2)
W, LowLP2 = sig.freqz(b, a, N)
s, LowLP2 = sig.freqs(B, A, fs*W)

# get transfer function of HIGH frequency driver's bandpass
# highpass
B, A, b, a = biquad_hp2nd(fHigh_HP, fs, bi1, ai1)
W, HighHP1 = sig.freqz(b, a, N)
s, HighHP1 = sig.freqs(B, A, fs*W)
B, A, b, a = biquad_hp2nd(fHigh_HP, fs, bi2, ai2)
W, HighHP2 = sig.freqz(b, a, N)
s, HighHP2 = sig.freqs(B, A, fs*W)
# lowpass
B, A, b, a = biquad_lp2nd(fHigh_LP, fs, bi1, ai1)
W, HighLP1 = sig.freqz(b, a, N)
s, HighLP1 = sig.freqs(B, A, fs*W)
B, A, b, a = biquad_lp2nd(fHigh_LP, fs, bi2, ai2)
W, HighLP2 = sig.freqz(b, a, N)
s, HighLP2 = sig.freqs(B, A, fs*W)

f = fs*W / (2.*np.pi)
# series connection of four 2nd order filters:
Low = (LowHP1*LowHP2) * (LowLP1*LowLP2)  
High = (HighHP1*HighHP2) * (HighLP1*HighLP2)

Low[0] = 1e-15  # avoid zero at DC
High[0] = 1e-15  # avoid zero at DC

# simulation of the acoustic summation on both driver's middle axis:
AcousticSum = Low + High  

#plot with pyplot-interface
plt.figure(figsize = (16,9))
#plt.xkcd()  # comic style
plt.semilogx(f, 20*np.log10(np.abs(Low)), 'C0',
             label = r'electric bandpass for low frequency driver')
plt.semilogx(f, 20*np.log10(np.abs(High)), 'C1', 
             label = r'electric bandpass for high frequency driver')
plt.semilogx(f, 20*np.log10(np.abs(AcousticSum)), 'C2', 
             label = r'acoustic on-axis summation')
plt.autoscale("tight")
plt.title(r'4th-order Linkwitz-Riley X-Over for a Two-Way Loudspeaker')
plt.xlabel(r'$f$ / Hz')
plt.ylabel(r'20 log10 |$H$| / dB')
plt.axis([10, 20000, -30, +3])
plt.yticks(np.arange(-30, 3+3, 3));
plt.xticks((10, 20, 50, 100, 200, 500, 
            1000, 2000, 5000, 10000, 20000), 
           ["10", "20", "50", "100", "200", "500", 
            "1k", "2k", "5k", "10k", "20k"])
plt.xlim(10, 20000)
plt.ylim(-30, 3)
plt.grid(True, which="both", axis="both", 
         linestyle="-", linewidth=0.5, color=(0.8, 0.8, 0.8)) 
plt.legend(loc="best");
plt.show()

## Exercise

What filter characteristics has the acoustic sum?

To explore that in detail, plot and discuss the phase of the Linwitz-Riley filters as well as the acoustic sum.

Why sum the magnitudes of the low and the high frequency driver precisely to 0 dB at fc for the chosen example?

Is this also possible with Butterworth filters?

# References

[<a id="cit-Moschytz1981" href="#call-Moschytz1981">Moschytz1981</a>] G. S., ``_Active filter design handbook: For use with programmable pocket calculators and minicomputers_'',  1981.

[<a id="cit-TietzeSchenk2002" href="#call-TietzeSchenk2002">TietzeSchenk2002</a>] Ulrich Tietze and Christoph Schenk, ``_Halbleiter-Schaltungstechnik_'',  2002.

[<a id="cit-Zoelzer2002" href="#call-Zoelzer2002">Zoelzer2002</a>] Udo Zoelzer, ``_DAFX - Digital Audio Effects_'',  2002.

[<a id="cit-Zoelzer2005" href="#call-Zoelzer2005">Zoelzer2005</a>] Udo Zoelzer, ``_Digitale Audiosignalverarbeitung_'',  2005.

[<a id="cit-Zoelzer2008" href="#call-Zoelzer2008">Zoelzer2008</a>] Udo Zoelzer, ``_Digitale Audiotechnik: Signalverarbeitung, Filter und Effekte_'',  2008.

[<a id="cit-Zoelzer2011" href="#call-Zoelzer2011">Zoelzer2011</a>] Udo Zoelzer, ``_DAFX - Digital Audio Effects_'',  2011.

[<a id="cit-Kimball1938" href="#call-Kimball1938">Kimball1938</a>] H. Kimball, ``_Attenuation Equalizers_'',  1938.

[<a id="cit-Bristow-Johnson" href="#call-Bristow-Johnson">Bristow-Johnson</a>] R. Bristow-Johnson, ``_Cookbook formulae for audio EQ biquad filter coefficients_'',  .  [online](http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt)

[<a id="cit-Bristow-Johnson1994" href="#call-Bristow-Johnson1994">Bristow-Johnson1994</a>] R. Bristow-Johnson, ``_The equivalence of various methods of computing biquad coefficients for audio parametric equalizers_'', Proc. of 97th AES Convention, San Fransisco,  1994.

[<a id="cit-Orfanidis1997" href="#call-Orfanidis1997">Orfanidis1997</a>] J. Sophoncles, ``_Digital parametric equalizer design with prescribed Nyquist-frequency gain_'', J. Aud. Eng. Soc., vol. 45, number 6, pp. 444-455,  1997.

[<a id="cit-Christensen2003" href="#call-Christensen2003">Christensen2003</a>] K. Bank, ``_A Generalization of the Biquadratic Parametric Equalizer_'', Proc. of 115th AES Convention, New York,  2003.

[<a id="cit-OppenheimSchaferBuck2004" href="#call-OppenheimSchaferBuck2004">OppenheimSchaferBuck2004</a>] Alan V., Ronald W. and John R., ``_Zeitdiskrete Signalverarbeitung_'',  2004.

[<a id="cit-Clark2000" href="#call-Clark2000">Clark2000</a>] J. Rob, ``_Techniques for Generating Digital Equalizer Coefficients_'', J. Aud. Eng. Soc., vol. 48, number 4, pp. 281-298,  2000.

[<a id="cit-Gunness2007" href="#call-Gunness2007">Gunness2007</a>] D. W., ``_Optimizing the Magnitude Response of Matched Z-Transform Filters ("MZTi") for Loudspeaker Equalization_'', Proc. of the 32nd International AES Conference: DSP For Loudspeakers, Hilleroed, Denmark,  2007.

[<a id="cit-Alaoui2007" href="#call-Alaoui2007">Alaoui2007</a>] Adnan Mohamad, ``_Novel Approach to Analog-to-Digital Transforms_'', IEEE Transactions on Circuits and Systems I: Regular Papers, vol. 54, number 2, pp. 338-350,  2007.

[<a id="cit-Schmidt2010" href="#call-Schmidt2010">Schmidt2010</a>] T. Schmidt, ``_Digital equalization filter: New solution to the frequency response near Nyquist and evaluation by listening tests_'', Proc. of 128th AES Convention, London,  2010.

[<a id="cit-Alaoui2010" href="#call-Alaoui2010">Alaoui2010</a>] Adnan Mohamad, ``_Improving the Magnitude Responses of Digital Filters for Loudspeaker Equalization_'', J. Aud. Eng. Soc., vol. 58, number 12, pp. 1064-1082,  2010.

[<a id="cit-Thaden1997" href="#call-Thaden1997">Thaden1997</a>] Rainer Thaden, ``_Entwicklung und Erprobung einer digitalen parametrischen Filterbank_'',  1997.



**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Sascha Spors, Digital Signal Processing - Lecture notes featuring computational examples, 2016-2018*.